# Invoking Streamlit App through ngrok tunnel

In [7]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11
!pip install langchain-chroma==0.1.4
!pip install sentence-transformers==2.7.0

%%capture
!pip install unstructured selenium textstat tiktoken
!pip install pyngrok==7.1.5
!pip install streamlit==1.32.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.6 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 24.0 MB/s eta 

UsageError: Line magic function `%%capture` not found.


In [8]:
%%capture
!pip install openai unstructured selenium textstat tiktoken


In [9]:
!pip install streamlit

In [ ]:
!pip install pyngrok==7.1.5

In [ ]:
##%%writefile app.py

Overwriting app.py


In [10]:
import streamlit as st

In [11]:
import tiktoken
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
#from textstat import flesch_reading_ease
from collections import Counter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader, SeleniumURLLoader
from langchain_chroma import Chroma
#from langchain_community.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from langchain_core.callbacks.base import BaseCallbackHandler
from operator import itemgetter
import streamlit as st
import tempfile
import os
import pandas as pd
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

## Load Open AI API Credentials

In [12]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [13]:
# import yaml

# with open('chatgpt_api_credentials.yml', 'r') as file:
#     api_creds = yaml.safe_load(file)

# api_creds.keys()

dict_keys(['openai_key'])

In [14]:
# import os

# os.environ['OPENAI_API_KEY'] = api_creds['openai_key']
# OPENAI_KEY = api_creds['openai_key']

In [ ]:
#from getpass import getpass
#OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


In [ ]:
# Customize initial app landing page

# st.set_page_config(page_title="File QA Chatbot", page_icon="🤖")
# st.title("Welcome to File QA RAG Chatbot 🤖")

# print("hi)")

Overwriting app.py


In [ ]:
website = "https://my.clevelandclinic.org/health/diseases/6424-asthma"

### START Streamlit app

In [16]:

!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11
!pip install langchain-chroma==0.1.4
!pip install sentence-transformers==2.7.0


  Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.20.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Succe

In [17]:
%%capture
!pip install openai unstructured selenium textstat tiktoken

In [18]:
%%capture
!pip install unstructured selenium textstat tiktoken
!pip install pyngrok==7.1.5
!pip install streamlit==1.32.2

In [ ]:
#!pip install streamlit

#!pip install pyngrok==7.1.5

In [ ]:
#!streamlit run app.py --server.port=8989 &>./logs.txt &

### ngrok URL

https://dashboard.ngrok.com/get-started/your-authtoken

In [ ]:
# from pyngrok import ngrok
# import yaml

# # Terminate open tunnels if exist
# ngrok.kill()

# # Setting the authtoken
# # Get your authtoken from `ngrok_credentials.yml` file
# with open('ngrok_credentials.yml', 'r') as file:
#     NGROK_AUTH_TOKEN = yaml.safe_load(file)
# ngrok.set_auth_token(NGROK_AUTH_TOKEN['ngrok_key'])

# # Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
# ngrok_tunnel = ngrok.connect(8989)
# #ngrok_tunnel = ngrok.connect(8501)
# print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://c8be-34-44-41-65.ngrok-free.app


In [ ]:
!pip install streamlit
!pip install pyngrok==7.1.5


In [ ]:
import streamlit as st

In [ ]:
!lsof -i :8989

COMMAND     PID USER   FD   TYPE DEVICE SIZE/OFF NODE NAME
streamlit 26826 root    6u  IPv4 714970      0t0  TCP *:8989 (LISTEN)
streamlit 26826 root    7u  IPv6 714971      0t0  TCP *:8989 (LISTEN)
streamlit 26826 root    9u  IPv4 719848      0t0  TCP localhost:8989->localhost:42740 (ESTABLISHED)
ngrok     27368 root    9u  IPv4 718771      0t0  TCP localhost:42740->localhost:8989 (ESTABLISHED)


In [ ]:
!kill $(lsof -t -i:8989)

In [ ]:

# """### START Streamlit app"""

# !streamlit run app.py --server.port=8989 &>./logs.txt &

In [ ]:
# Terminate open tunnels if exist
# ngrok.kill()

In [ ]:
from pyngrok import ngrok
import yaml


In [5]:
!pip install streamlit
!pip install pyngrok==7.1.5
!pip install psutil
!pip install openai
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.8 MB/s eta 0:00:00


# MAIN CODE TO RUN STREAMLIT APP

In [26]:

import streamlit as st
from pyngrok import ngrok
import yaml
import os
import time
import requests
import psutil
import signal
from datetime import datetime
from openai import OpenAI

def log_to_colab(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] {message}")

# First, let's verify the API credentials
try:
    # Check OpenAI credentials
    with open("chatgpt_api_credentials.yml", "r") as file:
        api_creds = yaml.safe_load(file)
        OPENAI_KEY = api_creds["openai_key"]
    log_to_colab("OpenAI credentials loaded")

    # Check ngrok credentials
    with open("ngrok_credentials.yml", "r") as file:
        ngrok_creds = yaml.safe_load(file)
        NGROK_TOKEN = ngrok_creds["ngrok_key"]
    log_to_colab("Ngrok credentials loaded")
except Exception as e:
    log_to_colab(f"Error loading credentials: {str(e)}")
    raise e

# Test OpenAI API
def test_openai():
    try:
        client = OpenAI(api_key=OPENAI_KEY)
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": "Hello!"}]
        )
        log_to_colab("OpenAI API test successful")
        return True
    except Exception as e:
        log_to_colab(f"OpenAI API test failed: {str(e)}")
        return False

# Port management functions
def kill_process_on_port(port):
    try:
        if os.name == 'posix':
            os.system(f'lsof -ti:{port} | xargs kill -9')
        else:
            os.system(f'for /f "tokens=5" %a in (\'netstat -aon ^| find ":{port}" ^| find "LISTENING"\') do taskkill /f /pid %a')
        time.sleep(2)
        log_to_colab(f"Killed process on port {port}")
    except Exception as e:
        log_to_colab(f"Error killing process on port {port}: {e}")

def is_port_available(port):
    import socket
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        sock.bind(('localhost', port))
        sock.close()
        return True
    except:
        return False

# Main execution
try:
    # 1. Test OpenAI API
    if not test_openai():
        raise Exception("OpenAI API test failed")

    # 2. Kill existing processes
    ngrok.kill()
    log_to_colab("Killed existing ngrok processes")

    # 3. Set up port
    port = 8501  # Using Streamlit's default port
    kill_process_on_port(port)

    if not is_port_available(port):
        log_to_colab(f"Port {port} is not available, trying alternative...")
        port = 8502
        if not is_port_available(port):
            raise Exception("No available ports found")

    log_to_colab(f"Using port {port}")

    # 4. Start Streamlit
    log_to_colab("Starting Streamlit...")
    os.system(f'streamlit run app.py --server.port={port} &>./logs.txt &')
    time.sleep(5)  # Give Streamlit time to start

    def monitor_logs():
      #"""Monitor both Streamlit and app logs"""
      try:
          print("\nStreamlit logs:")
          !tail -f logs.txt &
          print("\nChatbot debug logs:")
          !tail -f chatbot_debug.log &
      except Exception as e:
        print(f"Error monitoring logs: {e}")

    # 5. Set up ngrok
    ngrok.set_auth_token(NGROK_TOKEN)
    log_to_colab("Ngrok authentication successful")

    # 6. Create tunnel
    ngrok_tunnel = ngrok.connect(port)
    log_to_colab(f"Streamlit App URL: {ngrok_tunnel.public_url}")



    # 7. Monitor logs
    log_to_colab("Starting log monitoring...")
    !tail -f logs.txt &
    !tail -f chatbot_debug.log &

except Exception as e:
    log_to_colab(f"Setup failed: {str(e)}")
    raise e


monitor_logs()


# Add this to test the retriever directly
def test_retriever():
    try:
        test_query = "What is asthma?"
        docs = final_retriever.invoke(test_query)
        print(f"Retrieved {len(docs)} documents")
        print("First document preview:", docs[0].page_content[:100])
    except Exception as e:
        print(f"Retriever test failed: {e}")

# Test the retriever
test_retriever()

!tail -f chatbot_debug.log

# Keep the notebook running
while True:
    try:
        time.sleep(1)
        # Check if Streamlit is still running
        response = requests.get(f"http://localhost:{port}")
    except KeyboardInterrupt:
        log_to_colab("Shutting down...")
        ngrok.kill()
        break
    except:
        pass

[2025-01-19 15:32:08] OpenAI credentials loaded
[2025-01-19 15:32:08] Ngrok credentials loaded
[2025-01-19 15:32:09] OpenAI API test successful
[2025-01-19 15:32:09] Killed existing ngrok processes
[2025-01-19 15:32:11] Killed process on port 8501
[2025-01-19 15:32:11] Using port 8501
[2025-01-19 15:32:11] Starting Streamlit...
[2025-01-19 15:32:16] Ngrok authentication successful
[2025-01-19 15:32:17] Streamlit App URL: https://2e52-35-197-152-60.ngrok-free.app
[2025-01-19 15:32:17] Starting log monitoring...


KeyboardInterrupt: 

# DEBUGGING to see the log messages

In [20]:
# Add this to your Colab notebook
# Display the contents of the log file
#!cat chatbot_debug.log

# Or to see the last few lines
!tail -n 50 chatbot_debug.log

# To continuously monitor the log file (useful while using the chatbot)
##!tail -f chatbot_debug.log

2025-01-19 14:42:18,241 - DEBUG - in-event <InotifyEvent: src_path=b'/content/chatbot_debug.log', wd=1, mask=IN_MODIFY, cookie=0, name='chatbot_debug.log'>
2025-01-19 14:42:18,241 - DEBUG - in-event <InotifyEvent: src_path=b'/content/chatbot_debug.log', wd=1, mask=IN_MODIFY, cookie=0, name='chatbot_debug.log'>
2025-01-19 14:42:18,241 - DEBUG - in-event <InotifyEvent: src_path=b'/content/chatbot_debug.log', wd=1, mask=IN_MODIFY, cookie=0, name='chatbot_debug.log'>
2025-01-19 14:42:18,242 - DEBUG - in-event <InotifyEvent: src_path=b'/content/chatbot_debug.log', wd=1, mask=IN_MODIFY, cookie=0, name='chatbot_debug.log'>
2025-01-19 14:42:18,242 - DEBUG - in-event <InotifyEvent: src_path=b'/content/chatbot_debug.log', wd=1, mask=IN_MODIFY, cookie=0, name='chatbot_debug.log'>
2025-01-19 14:42:18,242 - DEBUG - in-event <InotifyEvent: src_path=b'/content/chatbot_debug.log', wd=1, mask=IN_MODIFY, cookie=0, name='chatbot_debug.log'>
2025-01-19 14:42:18,242 - DEBUG - in-event <InotifyEvent: src_pa

In [ ]:
# !cat chatbot_debug.log

start here